In [1]:
import pandas as pd
import numpy as np
import sys

In [2]:
input_dir = "/mnt/e/Science/ssHiC/Contact_files/Sparse/S288c/" 
output_dir = "/mnt/d/Science/Owncloud/Experiments/Capture/Analysis/Enrichment/"

samples=["AD162", "AD163", "AD162", "AD163", "AD164", "AD165", "AD166", "AD167", "AD168", "AD169", "AD206", "AD207", 
           "AD208", "AD209", "AD210", "AD211", "AD212", "AD213", "AD233", "AD234", "AD235", "AD236", 
           "AD237", "AD238", "AD239", "AD240", "AD241", "AD242", "AD243", "AD244", "AD245", "AD246", 
           "AD247", "AD248", "AD257", "AD258", "AD259", "AD260", "AD289", "AD290", "AD291", "AD292", "AD293", "AD294", "AD295", "AD296", "AD297", "AD298", "AD299",
         "AD300", "AD301", "AD302"]


probes_dsDNA_pos = [5315, 8579, 30750, 32542, 38864, 65930, 68339]
probes_dsDNA_name = ["chr2-650593-MET8", "chr4-64420-CDC13", "chr8-140404-ARG4-54nt",
                     "chr8-428677-DNA2", "chr10-449622-POL31", "chr15-620339-PDR5", "chr15-996452-MEK1"]
probes_dsDNAneg_pos = [2630, 9403, 30534, 64173]
probes_dsDNAneg_name = ["Neg_chr2-199707", "Neg_chr4-200383", "Neg_chr8-103079", "Neg_chr15-331170"]
probes_ssDNA_pos = [18529, 18535, 18559, 18583, 18589, 18605, 18611, 18614, 18616, 18621, 18632,
                   18634, 18666, 18673, 18676, 18694, 18706]
probes_ssDNA_name = ["Probe_URA-L-17213-MfeI-RC-/-Probe_URA-L-16220-MfeI-RC", "Probe_URA-L-15683-SspI-RC",
                    "Probe_URA-L-9924-MfeI-RC", "Probe_URA-L-6532-MfeI-RC", "Probe_URA-L-6065-SspI-RC",
                    "Probe_URA-L-3728-SspI-RC", "Probe_URA-L-3081-MfeI-RC", "Probe_URA-L-2599-MfeI-RC",
                    "Probe_URA-L-1560-SspI-RC", "Probe_URA-R-1439-SspI", "Probe_URA-R-2715-93-SspI",
                    "Probe_URA-R-2954-SspI", "Probe_URA-R-8073-SspI", "Probe_URA-R-9028-MfeI",
                    "Probe_URA-R-10241-SspI", "Probe_URA-R-12116-SspI", "Probe_URA-R-13482-SspI-/-Probe_URA-R-14040-MfeI"]
probes_ssDNAneg_pos = [18600, 18618]
probes_ssDNAneg_name = ["Probe_URA-L-4543-SspI-RC", "Probe_URA-L-774-MfeI-RC"]


probes_name = np.concatenate((probes_dsDNA_name, probes_dsDNAneg_name, probes_ssDNA_name, probes_ssDNAneg_name))
probes_pos = np.concatenate((probes_dsDNA_pos, probes_dsDNAneg_pos, probes_ssDNA_pos, probes_ssDNAneg_pos))

Script first removes the fragments corresponding to captured probes (both ssDNA and dsDNA) specified above in both the fragments file and the contact file. Then it draws 5000 random positions in the fragment file, and calculate how much contacts are made over all other fragments. the sum of all contacts is then divided by 5000. It gives the average number of contacts per fragment.

It then sum the number of contacts made per probe (summary_abs) and the enrichment over the average number of contacts on unenriched probes.

In [3]:
allsamples_abs= np.empty((0,len(probes_pos)+1))
allsamples_rel= np.empty((0,len(probes_pos)+1))
frags = pd.read_csv(str(input_dir + "fragments_list_S288c_DSB_LY_Capture_artificial_DpnIIHinfI.txt"), sep = "\t")
#Eliminate position of specified dsDNA and ssDNA probes
frags_filtered = frags.drop(np.concatenate((probes_dsDNA_pos, probes_ssDNA_pos)))
#Select 5000 random positions in the fragment list lacking the ssDNA+dsDNA probes positions to screen contacts from 
fragments=np.random.randint(0,len(frags_filtered.index),5000)

for sample in samples:
    summary=[]
    summary_abs = []
    summary_rel = []
    sparse_mat = pd.read_csv(str(input_dir + sample + "_S288c_DSB_LY_Capture_artificial_cutsite_PCRdupkept_q30.txt"), header=0, sep="\t", names=['Frag_A','Frag_B','Contacts'])
    #Eliminate rows containing specified dsDNA and ssDNA probes and calculate average number of contacts with each
    #fragments in the genome
    contacts_neg = []
    for i in np.concatenate((probes_dsDNA_pos, probes_ssDNA_pos)):
        sparse_mat_filtered = sparse_mat[ (sparse_mat['Frag_A'] != i) & (sparse_mat['Frag_B'] != i)]
    for a in fragments:
        b = sparse_mat_filtered[ (sparse_mat_filtered['Frag_A'] == a) | (sparse_mat_filtered['Frag_B'] == a)]
        contacts_neg.append(sum(b['Contacts']))
    summary_abs.append(sum(contacts_neg)/5000)
    summary_rel.append(1)
    for probe in probes_pos:
        c = sparse_mat[ (sparse_mat['Frag_A'] == probe) | (sparse_mat['Frag_B'] == probe)]
        summary_abs.append(sum(c['Contacts']))
        summary_rel.append(sum(c['Contacts'])/(sum(contacts_neg)/5000))
    #print(summary)
    label = np.concatenate((["random5000"], probes_name))
    export=np.array([label, summary_abs, summary_rel])
    #allsamples_abs=np.stack((allsamples_abs,summary_abs), axis=0)
    #allsamples_rel=np.stack((allsamples_rel,summary_rel), axis=0)
    summary=np.array([label, summary_abs, summary_rel])
    summary=np.transpose(summary)
    np.savetxt(str(output_dir + str(sample) + "_Probes_Enrichment_PCRdupkept.tsv"), summary, delimiter='\t', fmt="%s")    

#allsamples_abs=np.array((samples, allsamples_abs))
#allsamples_abs=np.transpose(allsamples_bas)

#allsamples_rel=np.array((samples, allsamples_rel))
#allsamples_rel=np.transpose(allsamples_rel)
#np.savetxt(str(output_dir + "All_samples_Absolute_contacts_per_probes.tsv"), allsamples_abs, delimiter='\t', fmt="%s")    
#np.savetxt(str(output_dir + "All_samples_Enrichment_per_probes.tsv"), allsamples_rel, delimiter='\t', fmt="%s")    
